# Introduction:
This notebook enables a step wise reproduction of the study presented in the paper:

> **Interpretation of Intracardiac Electrograms Through Textual Representations**
> William Jongwon Han, Diana Gomez, Avi Alok, Chaojing Duan, Michael A. Rosenberg, Douglas Weber, Emerson Liu, Ding Zhao
arXiv:2402.01115
Accepted at CHIL 2024


# Notebook Overview:

This notebook is designed to replicate the experiments and analyses conducted in the above paper within Colab, however it can also be executed in a similar captive environment.

It includes:


*  Data Preprocessing: Fetching loading and preparing the Intracardiac Atrial Fibrillation Database for model training.

*  Training: Fine-tuning 4 masked language models on the processed EGM data.

*  Inference: Assessing model performance based on MSE, MAE and AFib classification tasks.

*  Visualization: Visualizing, the reconstructed signal, attribution and attention scores to understand model inference.


By following this notebook, you can explore the innovative methodology proposed in the paper and gain hands-on experience with the techniques discussed.

**Limitation:** While the paper evaluates performance on both internal and external datasets, only the external dataset is publicly available due to the sensitive nature of clinical data. The public dataset contains only a single (positive) class, which may introduce bias or limit the reproducibility and generalization of the results. Users replicating the study should be aware that this class imbalance can significantly affect both training and evaluation metrics especially the classification metrics.

### Clone this repository

In [ ]:
!git clone https://github.com/spot-p/Text-EGM.git

### Move into the main directory and install dependencies.

In [2]:
%cd Text-EGM
!pip install -r colab_requirements.txt

/content/Text-EGM


## Data PreProcessing:

### Download the external dataset from physionet.org

In [ ]:
%cd preprocess
#!wget https://physionet.org/static/published-projects/iafdb/intracardiac-atrial-fibrillation-database-1.0.0.zip
!gdown 'https://drive.google.com/uc?export=download&id=1XYnKKhNOMuLk5madEJLH-G4YISidu0R-'
!unzip intracardiac-atrial-fibrillation-database-1.0.0


### Execute the preprocessing script

In [ ]:
!python preprocess_intra.py --path intracardiac-atrial-fibrillation-database-1.0.0
%cd ..

## Training

### Train each of the 4 models with processed data

##### With L4 GPU the average execution time per epoch is 37 minutes

In [ ]:
!python train.py --device=cuda:0 --batch=4 --patience=5 --model=big --mask=0.75 --use_ce --norm_loss=0.1 --warmup=500 --epochs=20

In [ ]:
!python train.py --device=cuda:0 --batch=4 --patience=5 --model=clin_bird --mask=0.75 --use_ce --norm_loss=0.1 --warmup=500 --epochs=20

In [ ]:
!python train.py --device=cuda:0 --batch=4 --patience=5 --model=long --mask=0.75 --use_ce --norm_loss=0.1 --warmup=500 --epochs=20

In [ ]:
!python train.py --device=cuda:0 --batch=4 --patience=5 --model=clin_long --mask=0.75 --use_ce --norm_loss=0.1 --warmup=500 --epochs=20

## Inference
##### ** The inference metrics (MSE,MAE,Accuracy and derived) for the below combinations would be saved into results.txt in the root folder with associated arguments used.

#### Perform Inference without counter-factuals/ inputs perturbations. The --checkpoint arugment would be different if the above training arguments are changed.

In [ ]:
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_big_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=big --inference --mask=0.75


In [ ]:
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_clin_bird_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=clin_bird --inference --mask=0.75


In [ ]:
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_long_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=long --inference --mask=0.75


In [ ]:
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_clin_long_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=clin_long --inference --mask=0.75


#### Perform inference with combinations of counterfactuals / input perturbations.

In [ ]:
# All four with label flipping
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_long_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=long --inference --mask=0.75 --LF
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_clin_long_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=clin_long --inference --mask=0.75 --LF
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_clin_bird_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=clin_bird --inference --mask=0.75 --LF
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_big_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=big --inference --mask=0.75 --LF

# All four with token substitution
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_long_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=long --inference --mask=0.75 --TS
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_clin_long_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=clin_long --inference --mask=0.75 --TS
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_clin_bird_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=clin_bird --inference --mask=0.75 --TS
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_big_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=big --inference --mask=0.75 --TS

# All four with label flipping and token substitution
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_long_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=long --inference --mask=0.75 --LF --TS
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_clin_long_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=clin_long --inference --mask=0.75 --LF --TS
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_clin_bird_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=clin_bird --inference --mask=0.75 --LF --TS
!python inference.py --device=cuda:0 --batch=1 --checkpoint='saved_best_0.0001_4_5_0.01_big_True_0.75_1.0_1.0_False_0.1_False_False_False' --model=big --inference --mask=0.75 --LF --TS


## Visualization

In [ ]:
%cd visualize

##### To visualize the masked reconstructed signal for each model

In [ ]:
!python stitching.py --checkpoint='saved_best_0.0001_4_5_0.01_long_True_0.75_1.0_1.0_False_0.1_False_False_False' --instance=100 --time=1
!python stitching.py --checkpoint='saved_best_0.0001_4_5_0.01_long_True_0.75_1.0_1.0_False_0.1_False_False_False' --instance=100 --time=2

!python stitching.py --checkpoint='saved_best_0.0001_4_5_0.01_clin_long_True_0.75_1.0_1.0_False_0.1_False_False_False' --instance=100 --time=1
!python stitching.py --checkpoint='saved_best_0.0001_4_5_0.01_clin_long_True_0.75_1.0_1.0_False_0.1_False_False_False' --instance=100 --time=2

!python stitching.py --checkpoint='saved_best_0.0001_4_5_0.01_clin_bird_True_0.75_1.0_1.0_False_0.1_False_False_False' --instance=100 --time=1
!python stitching.py --checkpoint='saved_best_0.0001_4_5_0.01_clin_bird_True_0.75_1.0_1.0_False_0.1_False_False_False' --instance=100 --time=2

!python stitching.py --checkpoint='saved_best_0.0001_4_5_0.01_big_True_0.75_1.0_1.0_False_0.1_False_False_False' --instance=100 --time=1
!python stitching.py --checkpoint='saved_best_0.0001_4_5_0.01_big_True_0.75_1.0_1.0_False_0.1_False_False_False' --instance=100 --time=2

##### To visualize the token importance for reconstruction and forecast / attribution scores for each model. Switch out the respective checkpoints for attribution scores mapped across time and frequency domain.

In [ ]:
!python int_grad.py --checkpoint=saved_best_0.0001_5_3_0.01_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=long --afibmask
!python int_grad.py --checkpoint=saved_best_0.0001_5_3_0.01_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=long --pre --afibmask
!python int_grad.py --checkpoint=saved_best_0.0001_5_3_0.01_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=long
!python int_grad.py --checkpoint=saved_best_0.0001_5_3_0.01_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=long --pre

!python int_grad.py --checkpoint=saved_best_0.0001_4_3_0.01_clin_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_long --afibmask
!python int_grad.py --checkpoint=saved_best_0.0001_4_3_0.01_clin_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_long --pre --afibmask
!python int_grad.py --checkpoint=saved_best_0.0001_4_3_0.01_clin_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_long
!python int_grad.py --checkpoint=saved_best_0.0001_4_3_0.01_clin_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_long --pre

!python int_grad.py --checkpoint=saved_best_0.0001_4_5_0.01_clin_bird_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_bird --afibmask
!python int_grad.py --checkpoint=saved_best_0.0001_4_5_0.01_clin_bird_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_bird --pre --afibmask
!python int_grad.py --checkpoint=saved_best_0.0001_4_5_0.01_clin_bird_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_bird
!python int_grad.py --checkpoint=saved_best_0.0001_4_5_0.01_clin_bird_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_bird --pre

!python int_grad.py --checkpoint=saved_best_0.0001_4_5_0.01_big_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=big --afibmask
!python int_grad.py --checkpoint=saved_best_0.0001_4_5_0.01_big_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=big --pre --afibmask
!python int_grad.py --checkpoint=saved_best_0.0001_4_5_0.01_big_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=big
!python int_grad.py --checkpoint=saved_best_0.0001_4_5_0.01_big_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=big --pre


##### To generate attention scores mapped across time and frequency domain for respective models with perturbed inputs. Switch out the the respective model checkpoints for their respective attention scores.

In [ ]:
!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=long --afibmask
!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=long --pre --afibmask
!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=long
!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=long --pre

!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_clin_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_long --afibmask
!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_clin_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_long --pre --afibmask
!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_clin_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_long
!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_clin_long_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_long --pre

!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_clin_bird_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_bird --afibmask
!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_clin_bird_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_bird --pre --afibmask
!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_clin_bird_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_bird
!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_clin_bird_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=clin_bird --pre

!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_big_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=big --afibmask
!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_big_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=big --pre --afibmask
!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_big_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=big
!python viz_attentions.py --checkpoint=saved_best_0.0001_4_5_0.01_big_True_0.75_1.0_1.0_False_0.1_False_False_False --device=cuda:0 --model=big --pre
